In [89]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_absolute_error,mean_squared_error,f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import xgboost as gb
from sklearn.svm import SVC
# from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB

In [41]:
promo_test = pd.read_csv("test.csv")
promo_test.head()

,EmployeeNo,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,State_Of_Origin,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers
0,YAK/S/00005,Information Technology and Solution Support,First Degree or HND,Male,Agency and others,2,1976,7.5,2017,0,0,65,FCT,Yes,Married,No,No,1
1,YAK/S/00011,Information Technology and Solution Support,NaN,Male,Direct Internal process,2,1991,0.0,2018,0,0,69,OGUN,Yes,Married,No,No,1
2,YAK/S/00015,Research and Innovation,"MSc, MBA and PhD",Male,Direct Internal process,2,1984,7.5,2012,0,0,76,KANO,Yes,Married,No,No,1
3,YAK/S/00016,Customer Support and Field Operations,First Degree or HND,Male,Agency and others,2,1984,2.5,2009,0,0,52,RIVERS,No,Single,No,No,1
4,YAK/S/00017,Information Technology and Solution Support,First Degree or HND,Male,Agency and others,2,1983,7.5,2014,1,0,69,FCT,Yes,Married,No,No,0


In [42]:
promo_train = pd.read_csv("train.csv")
promo_train.head()

,EmployeeNo,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,State_Of_Origin,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers,Promoted_or_Not
0,YAK/S/00001,Commercial Sales and Marketing,"MSc, MBA and PhD",Female,Direct Internal process,2,1986,12.5,2011,1,0,41,ANAMBRA,No,Married,No,No,0,0
1,YAK/S/00002,Customer Support and Field Operations,First Degree or HND,Male,Agency and others,2,1991,12.5,2015,0,0,52,ANAMBRA,Yes,Married,No,No,0,0
2,YAK/S/00003,Commercial Sales and Marketing,First Degree or HND,Male,Direct Internal process,2,1987,7.5,2012,0,0,42,KATSINA,Yes,Married,No,No,0,0
3,YAK/S/00004,Commercial Sales and Marketing,First Degree or HND,Male,Agency and others,3,1982,2.5,2009,0,0,42,NIGER,Yes,Single,No,No,1,0
4,YAK/S/00006,Information and Strategy,First Degree or HND,Male,Direct Internal process,3,1990,7.5,2012,0,0,77,AKWA IBOM,Yes,Married,No,No,1,0


In [43]:
promo_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16496 entries, 0 to 16495
Data columns (total 18 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   EmployeeNo                           16496 non-null  object 
 1   Division                             16496 non-null  object 
 2   Qualification                        15766 non-null  object 
 3   Gender                               16496 non-null  object 
 4   Channel_of_Recruitment               16496 non-null  object 
 5   Trainings_Attended                   16496 non-null  int64  
 6   Year_of_birth                        16496 non-null  int64  
 7   Last_performance_score               16496 non-null  float64
 8   Year_of_recruitment                  16496 non-null  int64  
 9   Targets_met                          16496 non-null  int64  
 10  Previous_Award                       16496 non-null  int64  
 11  Training_score_average      

In [44]:
promo_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38312 entries, 0 to 38311
Data columns (total 19 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   EmployeeNo                           38312 non-null  object 
 1   Division                             38312 non-null  object 
 2   Qualification                        36633 non-null  object 
 3   Gender                               38312 non-null  object 
 4   Channel_of_Recruitment               38312 non-null  object 
 5   Trainings_Attended                   38312 non-null  int64  
 6   Year_of_birth                        38312 non-null  int64  
 7   Last_performance_score               38312 non-null  float64
 8   Year_of_recruitment                  38312 non-null  int64  
 9   Targets_met                          38312 non-null  int64  
 10  Previous_Award                       38312 non-null  int64  
 11  Training_score_average      

In [45]:
promo_train.describe()

,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,Promoted_or_Not
count,38312.000000,38312.000000,38312.000000,38312.000000,38312.000000,38312.000000,38312.000000,38312.000000
mean,2.253680,1986.209334,7.698959,2013.139695,0.352996,0.023152,55.366465,0.084595
std,0.609443,7.646047,3.744135,4.261451,0.477908,0.150388,13.362741,0.278282
min,2.000000,1950.000000,0.000000,1982.000000,0.000000,0.000000,31.000000,0.000000
25%,2.000000,1982.000000,5.000000,2012.000000,0.000000,0.000000,43.000000,0.000000
50%,2.000000,1988.000000,7.500000,2014.000000,0.000000,0.000000,52.000000,0.000000
75%,2.000000,1992.000000,10.000000,2016.000000,1.000000,0.000000,68.000000,0.000000
max,11.000000,2001.000000,12.500000,2018.000000,1.000000,1.000000,91.000000,1.000000


In [46]:
X = promo_train[['Last_performance_score', "Targets_met", "Previous_Award", "Training_score_average"]]
y = promo_train['Promoted_or_Not']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [47]:
print('X train rows:', X_train.shape)
print('y train rows:', y_train.shape)

X train rows: (30649, 4)
y train rows: (30649,)


In [48]:
model_lin = LogisticRegression(random_state=10)
model_lin.fit(X_train, y_train)
y_pred = model_lin.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      1.00      0.95      6991
           1       0.49      0.05      0.09       672

    accuracy                           0.91      7663
   macro avg       0.70      0.52      0.52      7663
weighted avg       0.88      0.91      0.88      7663



In [49]:
X = promo_train[['Last_performance_score', "Targets_met", "Previous_Award", "Training_score_average"]]
y = promo_train['Promoted_or_Not']

X_traina, X_testa, y_traina, y_testa = train_test_split(X,y, test_size=0.3, random_state=75)

In [50]:
s_model = SVC()
k_model = KNeighborsClassifier()

In [51]:
def model_fun (model, X_traina=X_traina, y_traina=y_traina, X_testa=X_testa):
    model.fit(X_traina, y_traina)
    y_preda = model.predict(X_testa)
    return y_preda

    

In [52]:
s_pred = model_fun(s_model)
print(classification_report(y_testa, s_pred))

              precision    recall  f1-score   support

           0       0.91      1.00      0.95     10494
           1       0.00      0.00      0.00      1000

    accuracy                           0.91     11494
   macro avg       0.46      0.50      0.48     11494
weighted avg       0.83      0.91      0.87     11494



C:\Users\user1\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\user1\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\user1\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [53]:
k_pred = model_fun(k_model)
print(classification_report(y_testa, k_pred))

              precision    recall  f1-score   support

           0       0.92      0.99      0.96     10494
           1       0.55      0.15      0.24      1000

    accuracy                           0.92     11494
   macro avg       0.74      0.57      0.60     11494
weighted avg       0.89      0.92      0.89     11494



In [54]:
X_val = promo_test[['Last_performance_score', "Targets_met", "Previous_Award", "Training_score_average"]]
# y_val = promo_test['Promoted_or_Not']



In [55]:
y_pred_new= k_model.predict(X_val)

In [56]:
y_pred_new

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [57]:
promo_test.head(1)

,EmployeeNo,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,State_Of_Origin,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers
0,YAK/S/00005,Information Technology and Solution Support,First Degree or HND,Male,Agency and others,2,1976,7.5,2017,0,0,65,FCT,Yes,Married,No,No,1


In [58]:
predicted_table = promo_test[["EmployeeNo"]]

In [59]:
predicted_table["Promoted_or_Not"] = y_pred_new

<ipython-input-59-0d27d004ba3f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predicted_table["Promoted_or_Not"] = y_pred_new


In [60]:
predicted_table.head(1)

,EmployeeNo,Promoted_or_Not
0,YAK/S/00005,0


## Exporting to kaggle

In [61]:
predicted_table.to_csv("yakub_predicted1.csv", index=False)

In [62]:
pd.read_csv("yakub_predicted1.csv")

,EmployeeNo,Promoted_or_Not
0,YAK/S/00005,0
1,YAK/S/00011,0
2,YAK/S/00015,0
3,YAK/S/00016,0
4,YAK/S/00017,0
...,...,...
16491,YAK/S/54797,0
16492,YAK/S/54798,0
16493,YAK/S/54800,0
16494,YAK/S/54803,0


# Tuning the Model KNN

In [63]:
kmodel_t = KNeighborsClassifier()

In [68]:
param_grid = {"leaf_size":list(range(1,20)),
              "n_neighbors":list(range(1,15)),
              "p":[1,2]}

In [69]:
grid = GridSearchCV(kmodel_t, param_grid, cv=3)

In [71]:
grid.fit(X_traina, y_traina)

GridSearchCV(cv=3, estimator=KNeighborsClassifier(),
             param_grid={'leaf_size': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                       13, 14, 15, 16, 17, 18, 19],
                         'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14],
                         'p': [1, 2]})

In [72]:
grid.best_params_

{'leaf_size': 3, 'n_neighbors': 13, 'p': 1}

In [ ]:
y_pred_k= kmodel.predict(X_val)

In [73]:
kmodel_k = KNeighborsClassifier(leaf_size=3,
                               n_neighbors=13,
                               p=1)

In [74]:
kmodel_k.fit(X_traina, y_traina)

KNeighborsClassifier(leaf_size=3, n_neighbors=13, p=1)

In [75]:
y_pred_newp = kmodel_k.predict(X_test)

In [76]:
print(classification_report(y_pred_newp, y_test))

              precision    recall  f1-score   support

           0       1.00      0.92      0.96      7563
           1       0.13      0.89      0.23       100

    accuracy                           0.92      7663
   macro avg       0.57      0.91      0.59      7663
weighted avg       0.99      0.92      0.95      7663



In [77]:
predicted_table2 = promo_test[["EmployeeNo"]]

In [79]:
y_pred_newv = kmodel_k.predict(X_val)

In [81]:
predicted_table2["Promoted_or_Not"] = y_pred_newv

<ipython-input-81-c44461e77ffe>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predicted_table2["Promoted_or_Not"] = y_pred_newv


In [82]:
predicted_table2.head()

,EmployeeNo,Promoted_or_Not
0,YAK/S/00005,0
1,YAK/S/00011,0
2,YAK/S/00015,0
3,YAK/S/00016,0
4,YAK/S/00017,0


In [83]:
predicted_table2.to_csv("yakub_tuned.csv", index=False)

In [84]:
pd.read_csv("yakub_tuned.csv")

,EmployeeNo,Promoted_or_Not
0,YAK/S/00005,0
1,YAK/S/00011,0
2,YAK/S/00015,0
3,YAK/S/00016,0
4,YAK/S/00017,0
...,...,...
16491,YAK/S/54797,0
16492,YAK/S/54798,0
16493,YAK/S/54800,0
16494,YAK/S/54803,0


In [88]:
print(f1_score(y_pred_newp,y_test))

0.2305699481865285


In [91]:
s_model.feature_importances_

AttributeError: 'SVC' object has no attribute 'feature_importances_'